### Create the mart layer of the air travel lakehouse

##### Declare the global variables

In [1]:
project_id = "cs378-fa2025"
region = "us-central1"
dataset = "air_travel_mrt"
region = "us-central1"

In [2]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset cs378-fa2025.air_travel_mrt


### User journey 1: I oversee the business development strategy for Starbucks. I am trying to understand airport traffic patterns so that I can figure out how to optimize Starbuck's presence at US airports.
#### Query strategy: join the tsa traffic data (TSA_Traffic) with the airport maps data (Airport_Concessions, Airport_Establishment) via the airport data (Airport).

#### Query 1: Get the average monthly traffic for Starbucks per airport across all years. The foot traffic is based on TSA data alone. Report is not specific to the traffic patterns of a terminal because the security checkpoint data in the TSA table is dirty and can't be joined to the terminal. Also, these numbers don't account for traffic coming from connecting flights.

In [5]:
%%bigquery
select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year

Query is running:   0%|          |

Downloading:   0%|          |

,month_of_year,airport_icao,airport_name,location,avg_foot_traffic
0,2,KAUS,Austin Bergstrom International Airport,Near Gate 14,422.20
1,11,KAUS,Austin Bergstrom International Airport,Near Gate 14,177.33
2,2,KBDL,Bradley International Airport,"Food court area, post-security",178.67
3,11,KBDL,Bradley International Airport,"Food court area, post-security",1.00
4,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",169.00
5,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",169.00
6,11,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",147.00
7,11,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00
8,2,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,455.80
9,2,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",455.80


##### Same query as before, but with more descriptive month and hour fields: month name and hour denoted as am/pm

In [6]:
%%bigquery
select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,airport_icao,airport_name,location,avg_foot_traffic
0,February,KAUS,Austin Bergstrom International Airport,Near Gate 14,422.20
1,November,KAUS,Austin Bergstrom International Airport,Near Gate 14,177.33
2,February,KBDL,Bradley International Airport,"Food court area, post-security",178.67
3,November,KBDL,Bradley International Airport,"Food court area, post-security",1.00
4,February,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",169.00
5,February,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",169.00
6,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",147.00
7,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00
8,February,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,455.80
9,February,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",455.80


##### Materialize the query results into a table in the mart dataset

In [7]:
%%bigquery
create or replace table air_travel_mrt.starbucks_airport_foot_traffic_by_month_year as
  select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

""


##### Check the mart

In [9]:
%%bigquery
select *
from air_travel_mrt.starbucks_airport_foot_traffic_by_month_year

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,airport_icao,airport_name,location,avg_foot_traffic
0,February,KSAV,Savannah Hilton Head International Airport,"Post-security, near Gate 5",132.00
1,February,KLAS,McCarran International Airport,Gate C,379.31
2,February,KPWM,Portland International Jetport Airport,Near Gate 6,240.00
3,February,KSFO,San Francisco International Airport,Level 1 Arrivals,215.36
4,November,KLAX,Los Angeles International Airport,Near Gate 27,7.60
5,February,KSAV,Savannah Hilton Head International Airport,"Pre-security, near Ticket Counters",132.00
6,November,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",142.00
7,February,KMYR,Myrtle Beach International Airport,"Level 1, near Gate A1",194.00
8,February,KBDL,Bradley International Airport,"Food court area, post-security",178.67
9,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00


##### Look at the same data from a weekday perspective across all months and years

In [11]:
%%bigquery
select extract(dayofweek from t.event_date) as weekday, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(dayofweek from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, weekday

Query is running:   0%|          |

Downloading:   0%|          |

,weekday,airport_icao,airport_name,location,avg_foot_traffic
0,2,KAUS,Austin Bergstrom International Airport,Near Gate 14,330.38
1,2,KBDL,Bradley International Airport,"Food court area, post-security",134.25
2,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",161.67
3,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",161.67
4,2,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,427.27
5,2,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",427.27
6,2,KFLL,Fort Lauderdale Hollywood International Airport,"Ticketing-Gates Level, near Gate D1",427.27
7,2,KLAS,McCarran International Airport,"Gate B, adjacent to the PGA Tour Grill",330.27
8,2,KLAS,McCarran International Airport,Gate C,330.27
9,2,KLAS,McCarran International Airport,Level 2 / Baggage Claim & Ticketing Level,330.27


In [13]:
%%bigquery
select distinct extract(dayofweek from event_date) as weekday
from air_travel_int.TSA_Traffic

Query is running:   0%|          |

Downloading:   0%|          |

,weekday
0,2


##### We only have data for one day of the week (Tuesday), so can't make a compelling report with the current data

##### Future work: 1) clean the security checkpoint data such that each checkpoint is labeled with its airport terminal, this will allow us to produce more granular terminal-level reports. 2) go back and relook at the raw pdfs to see why we are missing so much data

####

#### Question 2: Using the dining data extracted from terminal maps and enriched with the LLM, where are the most common and least common food items available for purchase at various US airports around the country?

In [ ]:
%%bigquery
select a.name as airport, menu_item, count(*) as count
from air_travel_int.Menu_Items m join air_travel_int.Business b
on m.business_name = b.name
join air_travel_int.Airport_Businesses ab on b.name = ab.business
join air_travel_int.Airport a on ab.icao = a.icao
group by a.name, menu_item
order by count(*) desc, airport
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,airport,menu_item,count
0,San Francisco International Airport,Salads,14
1,San Francisco International Airport,Sandwiches,13
2,Austin Bergstrom International Airport,Salads,9
3,Los Angeles International Airport,Salads,9
4,Los Angeles International Airport,Sandwiches,8
5,Los Angeles International Airport,Espresso,7
6,Los Angeles International Airport,Latte,7
7,San Francisco International Airport,Burgers,7
8,Austin Bergstrom International Airport,Sandwiches,6
9,Los Angeles International Airport,Burritos,6


##### Create the mart table from the previous query

In [ ]:
%%bigquery
create or replace table air_travel_mrt.airport_top_food_items as
  select a.name as airport, menu_item, count(*) as count
  from air_travel_int.Menu_Items m join air_travel_int.Business b
  on m.business_name = b.name
  join air_travel_int.Airport_Businesses ab on b.name = ab.business
  join air_travel_int.Airport a on ab.icao = a.icao
  group by a.name, menu_item
  order by count(*) desc, airport

Query is running:   0%|          |

""


### User journey 2: I am a customer experience advisor to LAX. I want to help my client figure out what are the airport food trends and dining options other airports are offering (i.e. competitive analysis)